In [2]:
# Imports
import pandas as pd
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.ensemble._forest import ExtraTreesClassifier

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.metrics import confusion_matrix, fbeta_score

from fairlearn.metrics import MetricFrame
from fairlearn.metrics import selection_rate, count
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import ExponentiatedGradient, ClassificationMoment, Moment, DemographicParity, GridSearch
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio, false_negative_rate_difference, false_positive_rate_difference, true_positive_rate_difference, true_negative_rate_difference, demographic_parity_ratio, demographic_parity_difference

# Suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
TEDS = "/Users/meghanareddy/Downloads/Teds_2019_cleaned_cri2_checkcorr.csv"
#fl = "/Users/minhthao/Desktop/MSDA/Spring 2023/SUD/FL_clean_final.csv"
#md = "/Users/alapa/Downloads/MD_clean_final.csv"

In [4]:
df = pd.read_csv(TEDS)

In [4]:
df.shape

(1722503, 38)

In [7]:
list(df.columns)

['PSYPROB',
 'SUB1',
 'DETNLF_D',
 'MARFLG',
 'SUB3',
 'FREQ_ATND_SELF_HELP_D',
 'PRIMINC',
 'EMPLOY_D',
 'DAYWAIT',
 'EDUC',
 'EMPLOY',
 'FRSTUSE1',
 'RACE',
 'SUB2',
 'SUB1_D',
 'OTHERFLG',
 'BARBFLG',
 'DETNLF',
 'GENDER',
 'DETCRIM',
 'SUB2_D',
 'MTHAMFLG',
 'METHUSE',
 'FREQ1_D',
 'PSOURCE',
 'DIVISION',
 'BENZFLG',
 'AGE',
 'MARSTAT',
 'PRIMPAY',
 'REASON',
 'DSMCRIT',
 'CBSA2010',
 'FREQ_ATND_SELF_HELP',
 'ALCDRUG',
 'FREQ2_D',
 'STFIPS',
 'L/S']

In [5]:
X = df.drop(columns = ["L/S"])
y = df["L/S"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Classification

In [10]:
# xgboost #Model Training
from sklearn.metrics import classification_report
clf = XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = pd.DataFrame(clf.predict(X_test),columns=["Passed"])
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.76      0.90      0.82    360825
           1       0.60      0.34      0.44    155926

    accuracy                           0.73    516751
   macro avg       0.68      0.62      0.63    516751
weighted avg       0.71      0.73      0.71    516751



In [11]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
test_scores = clf.predict_proba(X_test)[:, 1]
test_preds = (test_scores >= np.mean(y_train)) * 1
# Train AUC
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

0.765328896645966

In [12]:
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
print("F-1 score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("AUC:", auc)

F-1 score: 0.4352262824845874
Precision: 0.5960937848570026
Recall: 0.3427330913381989
Accuracy: 0.7315999388486911
AUC: 0.765328896645966


In [13]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

# Model Training
clf = LogisticRegression().fit(X_train, y_train)

# Model Evaluation
print("Accuracy: ", clf.score(X_test, y_test))

y_pred = pd.DataFrame(clf.predict(X_test), columns=["Passed"])
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

test_scores = clf.predict_proba(X_test)[:, 1]
test_preds = (test_scores >= np.mean(y_train)) * 1

# Train AUC
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

Accuracy:  0.6940944478094866
              precision    recall  f1-score   support

           0       0.71      0.95      0.81    360825
           1       0.47      0.10      0.16    155926

    accuracy                           0.69    516751
   macro avg       0.59      0.52      0.49    516751
weighted avg       0.64      0.69      0.62    516751



0.6523268741236219

In [14]:
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
print("F-1 score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("AUC:", auc)

F-1 score: 0.15960744076257719
Precision: 0.4665713486463805
Recall: 0.0962700255249285
Accuracy: 0.6940944478094866
AUC: 0.6523268741236219


In [15]:
# Random forest classification
from sklearn.ensemble import RandomForestClassifier

# Model Training
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0).fit(X_train, y_train)

# Model Evaluation
print("Accuracy: ", clf.score(X_test, y_test))

y_pred = pd.DataFrame(clf.predict(X_test), columns=["Passed"])
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

test_scores = clf.predict_proba(X_test)[:, 1]
test_preds = (test_scores >= np.mean(y_train)) * 1

# Train AUC
roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

Accuracy:  0.7631973619789802
              precision    recall  f1-score   support

           0       0.76      0.96      0.85    360825
           1       0.76      0.32      0.45    155926

    accuracy                           0.76    516751
   macro avg       0.76      0.64      0.65    516751
weighted avg       0.76      0.76      0.73    516751



0.8274123825545306

In [16]:
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
print("F-1 score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("AUC:", auc)

F-1 score: 0.4465991316931982
Precision: 0.7573703101512409
Recall: 0.3166630324641176
Accuracy: 0.7631973619789802
AUC: 0.8274123825545306


In [6]:
# Tabnet 
import pytorch_tabnet
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score

# Define and initialize TabNetClassifier model
tabnet_model = TabNetClassifier(optimizer_fn=torch.optim.Adam,
                                optimizer_params=dict(lr=2e-2),
                                scheduler_params={"step_size":10, "gamma":0.9},
                                scheduler_fn=torch.optim.lr_scheduler.StepLR,
                                mask_type='entmax')

# Train the model
# Remember to change to max epoch 50 with cleaned data
tabnet_model.fit(X_train.values, y_train.values,
                 eval_set=[(X_train.values, y_train.values), (X_test.values, y_test.values)],
                 eval_name=['train', 'valid'],
                 eval_metric=['auc', 'accuracy'],
                 max_epochs=50, patience=50,
                 batch_size=256, virtual_batch_size=128,
                 num_workers=0,
                 weights=1,
                 drop_last=False)

# Evaluate the trained model on the test set
y_pred_proba = tabnet_model.predict_proba(X_test.values)
y_pred = (y_pred_proba[:, 1] >= 0.5).astype(int) # thresholding at 0.5
auc = roc_auc_score(y_test, y_pred_proba[:, 1])
accuracy = accuracy_score(y_test, y_pred)

print(f"Test AUC: {auc:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

epoch 0  | loss: 0.54842 | train_auc: 0.80714 | train_accuracy: 0.69935 | valid_auc: 0.80599 | valid_accuracy: 0.69806 |  0:01:36s
epoch 1  | loss: 0.5166  | train_auc: 0.82562 | train_accuracy: 0.70737 | valid_auc: 0.82417 | valid_accuracy: 0.7056  |  0:03:09s
epoch 2  | loss: 0.50939 | train_auc: 0.82786 | train_accuracy: 0.72162 | valid_auc: 0.82611 | valid_accuracy: 0.71986 |  0:04:42s
epoch 3  | loss: 0.50443 | train_auc: 0.83192 | train_accuracy: 0.72444 | valid_auc: 0.83039 | valid_accuracy: 0.72297 |  0:06:15s
epoch 4  | loss: 0.50325 | train_auc: 0.83265 | train_accuracy: 0.72796 | valid_auc: 0.83032 | valid_accuracy: 0.7253  |  0:07:48s
epoch 5  | loss: 0.50421 | train_auc: 0.8349  | train_accuracy: 0.73594 | valid_auc: 0.83276 | valid_accuracy: 0.73358 |  0:09:40s
epoch 6  | loss: 0.49964 | train_auc: 0.83414 | train_accuracy: 0.74501 | valid_auc: 0.83142 | valid_accuracy: 0.74224 |  0:11:17s
epoch 7  | loss: 0.49833 | train_auc: 0.83723 | train_accuracy: 0.72058 | valid_auc

In [8]:
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
# auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
auc = roc_auc_score(y_test, y_pred_proba[:, 1])
print("F-1 score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("AUC:", auc)

F-1 score: 0.6489430288324215
Precision: 0.5634799533259952
Recall: 0.7649654323204597
Accuracy: 0.7502646342242202
AUC: 0.8424109771609885


In [9]:
# y_pred = pd.DataFrame(tabnet_model.predict(X_test), columns=["Passed"])
from sklearn.metrics import classification_report
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

# test_scores = tabnet_model.predict_proba(X_test)[:, 1]
# test_preds = (test_scores >= np.mean(y_train)) * 1

# # Train AUC
roc_auc_score(y_test, y_pred_proba[:, 1])



              precision    recall  f1-score   support

           0       0.88      0.74      0.81    360825
           1       0.56      0.76      0.65    155926

    accuracy                           0.75    516751
   macro avg       0.72      0.75      0.73    516751
weighted avg       0.78      0.75      0.76    516751



0.8424109771609885